#TensorFlow Mecanica 101

El objetivo de este tutorial es mostrar como utilizar TensorFlow para entrenar y evaluar una red neuronal feed-forward para la clasificacion de digitos escritos a mano mediante el dataset de MNIST. Se revisan dos archivos de python y se examinan las funciones principales de dichos archivos.

#Archivos usados en el Tutorial

En este tutorial se usan los siguientes dos archivos los cuales se pueden encontrar en el github del proyecto TensorFlow los cuales son:

* mnist.py (El codigo para construir un modelo MNIST plenamente conectado.)
* fully_connected_feed.py (El codigo principal para entrenar el modelo MNIST construido contra el conjunto de datos descargados usando un diccionario de alimentacion.)

Si se ejecuta el archivo fully_connected_feed.py se empieza a entrenar la red neuronal.

#Preparar los Datos

MNIST es un problema clasico de machine learning el cual consiste en un conjunto de imagenes de tamanio de 28 x 28 con digitos escritos a mano de 0-9 en lo cual se busca clasificar las imagenes de acuerdo a la clase a la que pertenecen por ejemplo si en la imagen hay un 0 se espera que se clasifique dicha imagen en la clase 0.

#Descarga

En el archivo fully_connected_feed.py hay una funcion que se llama run_training() en dicha funcion hay una linea de codigo que dice input_data.read_data_sets() la cual consiste en descargar los datos en una carpeta y regresa un diccionario el cual se llama data_sets en donde se encuentran las instancias a dichos datos.

data_sets = input_data.read_data_sets(FLAGS.train_dir, FLAGS.fake_data)

Los datos a tratar son los siguientes:

* data_sets.train (Contiene 55000 imagenes y etiquetas para entrenar)
* data_sets.validation (Contiene 5000 imagenes y etiquetas para validar la exactitud del entrenamiento)
* data_sets.test (Contiene 10000 imagenes y etiquetas para testear la exactitud del entrenamiento)


#Inputs y Placeholders

En el archivo fully_connected_feed.py hay una funcion que se llama placeholder_inputs() la cual crea dos ops tf.placeholders las cuales definen el tamanio de las entradas (inputs) incluyendo el tamanio del batch (batch_size) para el uso en el grafo en el cual se entrenaran los ejemplos y se les dara fed.

images_placeholder = tf.placeholder(tf.float32, shape=(batch_size, IMAGE_PIXELS))

labels_placeholder = tf.placeholder(tf.int32, shape=(batch_size))

Cuando se ejecute el loop de entrenamiento la imagen completa y las etiquetas del dataset se ajustan al tamanio del batch en cada iteracion ademas se emparejaran con las ops de placeholders y se pasaran dentro de una funcion de sesion tal como sess.run() esta funcion tendra como parametro a feed_dict.

#Construyendo el Grafo

Despues de crear los placegolders para los datos el grafo se construye a partir del archivo mnits.py de acuerdo a un patron de tres etapas las cuales son inference(), loss(), y training().

1. inference() - Construye el grafo en la medida que es requerido para ejecutar la red y hacer predicciones
2. loss() - Agrega las ops requeridas al grafo de inferencia para generar perdida.
3. training() - Agrega a el grafo de perdida las ops requeridas para calcular y aplicar los gradientes.

#Inferencia

La funcion de inference() construye el grafo en la medida que es requerido para regresar un tensor que contiene las salidas de las predicciones.

Toma las imagenes del placeholder como entrada y se basa en la parte superior de un par de capas completamente conectadas con la activacion Relu seguido de la capa lineal del nodo diez especificando la salida de logits (Inversa de la funcion logistica).

Cada capa se crea debajo de un unico tf.name_scope que actua como un prefijo a los objetos creados dendro de ese scope.

with tf.name_scope('hidden1') as scope:

Dentro del scope definido los pesos y bias que se usaran por cada una de estas capas son generados dentro de instancias a tf.Variable con los tamanios deseados.

weights = tf.Variable(tf.truncated_normal([IMAGE_PIXELS, hidden1_units], stddev=1.0 / math.sqrt(float(IMAGE_PIXELS))), name='weights')

biases = tf.Variable(tf.zeros([hidden1_units]), name='biases')

Para una instancia que se crea debajo de el scope de hidden1 el unico nombre que se le da a la variable pesos debe ser "hidden1/weights".

Cada variable se le da un inicializador de ops como parte de su construccion.

En los casos mas comunes los pesos son inicializados con la funcion tf.truncated_normal y se le da el tamanio del tensor 2-d la primera dimension representa el numero de unidades en la capa desde la cual los pesos estan conectados y la segunda dimension representa el numero de unidades en la capa hacia los pesos conectados. 

Para la primer capa con el nombre hidden1 las dimensiones son [IMAGE_PIXELS, hidden1_units] porque los pesos estan conectados a la entrada de la imagen de la capa hidden1. La funcion tf.truncated_normal genera e inicializa una distribucion aleatoria dada la media y la desviacion estandar.

Luego los biases son inicializados con tf.zeros para asegurarse que todos comienzan con valores de 0  y su tamanio es simplemente el numero de unidades en la capa en la cual estan conectados.

El grafo tiene tres ops primarias: dos ops tf.nn.relu de wrapping y tf.matmul para las capas ocultas ademas de una extra tf.matmul para los logits son creadas en cada turno con instancias separadas de tf.Variable conectadas a cada entrada de los placeholders o la salida de los tensores de la capa anterior.

hidden1 = tf.nn.relu(tf.matmul(images, weights) + biases)

hidden2 = tf.nn.relu(tf.matmul(hidden1, weights) + biases)

logits = tf.matmul(hidden2, weights) + biases

Finalmente el tensor de logits contendra la salida la cual se regresara.

#Perdida

La funcion loss() construye el grafo agregando las loss ops requeridas.

Primero los valores de los labels_placeholder estan codificados en tensores de 1-hot values. por ejemplo, si la clase identifica el valor '3' este se convertira en:

[0, 0, 0, 1, 0, 0, 0, 0, 0, 0]

batch_size = tf.size(labels)
labels = tf.expand_dims(labels, 1)
indices = tf.expand_dims(tf.range(0, batch_size, 1), 1)
concated = tf.concat(1, [indices, labels])
onehot_labels = tf.sparse_to_dense(concated, tf.pack([batch_size, NUM_CLASSES]), 1.0, 0.0)
    
La op tf.nn.softmax_cross_entropy_with_logits se agrega para comparar la salida de logits de la funcion inference() y las etiquetas 1-hot

cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits, onehot_labels, name='xentropy')

Si luego se usa tf.reduce_mean para obtener el promedio de cross-entropy a travez de la dimension del batch (La primera dimension) es la perdida total.

loss = tf.reduce_mean(cross_entropy, name='xentropy_mean')

Despues el tensor contendra el valor de perdida el cual se regresara.

Nota: La idea de cross-entropy viene de el campo de la teoria de la informacion que describe que tan mal son las predicciones de la red neuronal dado lo que actualmente es correcto. 

#Entrenamiento